In [1]:
# ============================================================
# v7_final_noocr_full.py
# 3090(24GB) 안정판: 정렬/정규화/진단 강화 + 5-Fold + TTA + Auto-weight (No OCR)
# - Stratified 5-Fold (공통 split)
# - EfficientNet-B3(380), ViT-B/16(384) 2-모델 앙상블
# - Mild Aug, LabelSmoothing, AMP(fp16), channels_last
# - GradCheckpoint(지원시), OOF/Test probs npz(float32) 저장
# - OOF 기반 Auto-weight → Test 앙상블 → 제출 + 진단 리포트
# ============================================================

import os, glob, json, random, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import cv2, timm, torch, albumentations as A
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

# ----------------------------
# Repro & Device
# ----------------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
if hasattr(torch, "set_float32_matmul_precision"):
    torch.set_float32_matmul_precision("high")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} ({torch.cuda.get_device_name(0) if device.type=='cuda' else 'CPU'})")

# ----------------------------
# Paths
# ----------------------------
BASE = "/data/ephemeral/home/data"
TRAIN_IMG_DIR = f"{BASE}/processed/stage0_6_train_v6/"
TEST_IMG_DIR  = f"{BASE}/processed/stage0_6_test_v6/"
TRAIN_META    = f"{BASE}/meta_stage0_6_train_v6.csv"
TRAIN_CSV     = f"{BASE}/raw/train.csv"
SUB_CSV       = f"{BASE}/raw/sample_submission.csv"

os.makedirs("./models", exist_ok=True)
os.makedirs("./oofs", exist_ok=True)
os.makedirs("./test_probs", exist_ok=True)
os.makedirs("./debug", exist_ok=True)

# ----------------------------
# Config
# ----------------------------
NUM_CLASSES   = 17
NFOLDS        = 5
EPOCHS        = 30
BATCH_SIZE    = 16          # 3090(24GB) 안전값
NUM_WORKERS   = 6
LR            = 1e-4
WEIGHT_DECAY  = 1e-4
LABEL_SMOOTH  = 0.05

BACKBONES = [
    dict(name="tf_efficientnet_b3_ns", img_size=380, drop_rate=0.4, drop_path=0.2, grad_ckpt=False),
    dict(name="vit_base_patch16_384",  img_size=384, drop_rate=0.2, drop_path=0.1, grad_ckpt=True),
]

# ----------------------------
# Utils
# ----------------------------
def safe_glob(base_dir, basename):
    cands = glob.glob(os.path.join(base_dir, "**", basename), recursive=True)
    return cands[0] if cands else None

def build_transforms(img_size):
    trn = A.Compose([
        A.Resize(img_size, img_size),
        A.Rotate(limit=10, border_mode=cv2.BORDER_REFLECT_101, p=0.4),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(0.15, 0.15, p=0.4),
        A.GaussNoise(var_limit=(5, 20), p=0.3),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    tst = A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    return trn, tst

def row_normalize(p):
    p = np.asarray(p, dtype=np.float64)
    p = np.nan_to_num(p, nan=0.0, posinf=0.0, neginf=0.0)
    p = np.clip(p, 1e-12, None)
    s = p.sum(axis=1, keepdims=True)
    s = np.where(s <= 0, 1.0, s)
    return p / s

def check_probs(name, probs):
    p = np.asarray(probs)
    msg = []
    if np.isnan(p).any(): msg.append("NaN")
    if np.isinf(p).any(): msg.append("Inf")
    if p.ndim != 2: msg.append(f"ndim={p.ndim}")
    if p.shape[1] != NUM_CLASSES: msg.append(f"num_classes={p.shape[1]}")
    row_sum = p.sum(axis=1)
    off = np.mean(np.abs(row_sum - 1.0))
    return f"[{name}] shape={p.shape}, rowSumMeanDelta={off:.3e}, flags={'|'.join(msg) if msg else 'OK'}"

def save_npz_float32(path, arr):
    arr = np.asarray(arr, dtype=np.float32)
    np.savez_compressed(path, arr)

def load_oof_array(path):
    if path.endswith(".npz") and os.path.exists(path):
        z = np.load(path)
        return z["arr_0"].astype(np.float32)
    if path.endswith(".npy") and os.path.exists(path):
        return np.load(path).astype(np.float32)
    raise FileNotFoundError(path)

def corrcoef(a, b):
    a = a.reshape(len(a), -1)
    b = b.reshape(len(b), -1)
    a = a - a.mean(axis=1, keepdims=True)
    b = b - b.mean(axis=1, keepdims=True)
    a_norm = np.sqrt((a**2).sum(axis=1, keepdims=True)) + 1e-12
    b_norm = np.sqrt((b**2).sum(axis=1, keepdims=True)) + 1e-12
    a = a / a_norm; b = b / b_norm
    return float(np.mean(np.sum(a * b, axis=1)))

# ----------------------------
# Dataset
# ----------------------------
class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(row["filepath"]).convert("RGB"))
        target = int(row["target"])
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, target

class ArrayDataset(Dataset):
    """Numpy 이미지 배열(TTA용)에 Albumentations 적용"""
    def __init__(self, arr_list, transform=None):
        self.arr = arr_list; self.transform = transform
    def __len__(self): return len(self.arr)
    def __getitem__(self, idx):
        img = self.arr[idx]
        if self.transform:
            img = self.transform(image=img)["image"]
        return img

# ----------------------------
# Model factory
# ----------------------------
def create_model(model_name, drop_rate=0.2, drop_path=0.1, grad_ckpt=False):
    model = timm.create_model(
        model_name, pretrained=True, num_classes=NUM_CLASSES,
        in_chans=3, drop_rate=drop_rate, drop_path_rate=drop_path
    )
    if grad_ckpt and hasattr(model, "set_grad_checkpointing"):
        model.set_grad_checkpointing(True)
    model = model.to(device)
    model = model.to(memory_format=torch.channels_last)
    return model

# ----------------------------
# Train / Valid
# ----------------------------
def train_one_epoch(loader, model, optimizer, loss_fn, scaler=None):
    model.train()
    total_loss, preds_list, targets_list = 0.0, [], []
    for images, targets in tqdm(loader, desc="Train", leave=False):
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
            logits = model(images)
            loss = loss_fn(logits, targets).mean()
        if scaler and device.type=="cuda":
            scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()
        total_loss += loss.item()
        preds_list.extend(logits.argmax(1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
    return {"loss": total_loss/len(loader),
            "acc": accuracy_score(targets_list, preds_list),
            "f1": f1_score(targets_list, preds_list, average="macro")}

@torch.no_grad()
def validate(loader, model, loss_fn):
    model.eval()
    total_loss, preds_list, targets_list = 0.0, [], []
    all_probs = []
    for images, targets in tqdm(loader, desc="Valid", leave=False):
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
            logits = model(images)
            loss = loss_fn(logits, targets).mean()
            probs = torch.softmax(logits, dim=1).to(torch.float32).cpu().numpy()
        total_loss += loss.item()
        all_probs.append(probs)
        preds_list.extend(probs.argmax(1))
        targets_list.extend(targets.cpu().numpy())
    all_probs = row_normalize(np.vstack(all_probs))
    return {"loss": total_loss/len(loader),
            "acc": accuracy_score(targets_list, preds_list),
            "f1": f1_score(targets_list, preds_list, average="macro"),
            "probs": all_probs, "targets": np.array(targets_list)}

def make_loader(df, transform, batch_size, shuffle, drop_last):
    ds = ImageDataset(df, transform)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                      num_workers=NUM_WORKERS, pin_memory=True,
                      drop_last=drop_last, persistent_workers=True)

# ----------------------------
# Load data & split
# ----------------------------
meta = pd.read_csv(TRAIN_META)
train_csv = pd.read_csv(TRAIN_CSV)
if "basename" not in meta.columns:
    meta["basename"] = meta["filepath"].apply(os.path.basename)
train_csv["basename"] = train_csv["ID"].apply(lambda x: f"{x}.jpg" if not str(x).endswith(".jpg") else x)
df = pd.merge(meta, train_csv[["basename","target"]], on="basename", how="left")
df = df.dropna(subset=["target"]).reset_index(drop=True)
df["target"] = df["target"].astype(int)

skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
SPLITS = list(skf.split(df, df["target"]))


Using cuda (NVIDIA GeForce RTX 3090)


In [2]:

# ----------------------------
# 5-Fold training per backbone → save OOF
# ----------------------------
all_backbone_fold_scores = {}

for bb in BACKBONES:
    model_name = bb["name"]; img_size = bb["img_size"]
    print(f"\n========== Backbone: {model_name} ({img_size}) ==========")
    trn_tf, tst_tf = build_transforms(img_size)
    fold_scores = []

    for fold, (trn_idx, val_idx) in enumerate(SPLITS):
        print(f"\n----- {model_name} | Fold {fold+1}/{NFOLDS} -----")
        trn_df, val_df = df.iloc[trn_idx].copy(), df.iloc[val_idx].copy()

        trn_df["filepath"] = trn_df["basename"].apply(lambda b: safe_glob(TRAIN_IMG_DIR, b))
        val_df["filepath"] = val_df["basename"].apply(lambda b: safe_glob(TRAIN_IMG_DIR, b))
        trn_df = trn_df.dropna(subset=["filepath"]).reset_index(drop=True)
        val_df = val_df.dropna(subset=["filepath"]).reset_index(drop=True)

        trn_loader = make_loader(trn_df, trn_tf, BATCH_SIZE, shuffle=True,  drop_last=True)
        val_loader = make_loader(val_df, tst_tf, BATCH_SIZE, shuffle=False, drop_last=False)

        model = create_model(model_name, bb["drop_rate"], bb["drop_path"], bb["grad_ckpt"])
        loss_fn = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH, reduction="none")
        optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
        scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

        best_f1 = -1.0; best_val = None
        best_path = f"./models/{model_name}_fold{fold}.pt"
        basenames_val = val_df["basename"].tolist()

        for epoch in range(EPOCHS):
            tr = train_one_epoch(trn_loader, model, optimizer, loss_fn, scaler)
            va = validate(val_loader, model, loss_fn)
            scheduler.step()
            print(f"[{model_name}][Fold {fold}] Ep {epoch+1}/{EPOCHS} | TrF1={tr['f1']:.4f} VaF1={va['f1']:.4f}")
            if va["f1"] > best_f1:
                best_f1 = va["f1"]; best_val = va
                torch.save(model.state_dict(), best_path)

        fold_scores.append(best_f1)
        save_npz_float32(f"./oofs/{model_name}_fold{fold}_probs.npz", best_val["probs"])
        np.save(f"./oofs/{model_name}_fold{fold}_targets.npy", best_val["targets"])
        np.save(f"./oofs/{model_name}_fold{fold}_basenames.npy", np.array(basenames_val))
        del model; torch.cuda.empty_cache()

    all_backbone_fold_scores[model_name] = fold_scores
    print(f">>> {model_name} fold F1: {fold_scores} | mean={np.mean(fold_scores):.4f}")

# ----------------------------
# Auto-weight (Eff + ViT) using OOF
# ----------------------------
def eval_weights(eff_list, vit_list, tgt_list, we, wv):
    y_pred, y_true = [], []
    for eff, vit, tgt in zip(eff_list, vit_list, tgt_list):
        mix = we*eff + wv*vit
        y_pred.append(mix.argmax(1))
        y_true.append(tgt)
    return f1_score(np.concatenate(y_true), np.concatenate(y_pred), average="macro")

eff_oof, vit_oof, tgt_oof = [], [], []
for fold in range(NFOLDS):
    eff_oof.append(load_oof_array(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_probs.npz"))
    # 호환: npz만 저장하지만 과거 npy가 있다면 아래 줄을 바꿔도 됨
    vit_oof.append(load_oof_array(f"./oofs/vit_base_patch16_384_fold{fold}_probs.npz"))
    tgt_oof.append(np.load(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_targets.npy"))

best_score, w_eff, w_vit = -1.0, 0.5, 0.5
for w in np.linspace(0, 1, 21):
    sc = eval_weights(eff_oof, vit_oof, tgt_oof, w, 1-w)
    if sc > best_score:
        best_score, w_eff, w_vit = sc, w, 1-w

print(f"\n[Auto-Weight OOF] best macro F1={best_score:.4f} | w_eff={w_eff:.2f}, w_vit={w_vit:.2f}")
with open("./oofs/best_weights.json", "w") as f:
    json.dump({"score": float(best_score), "w_eff": float(w_eff), "w_vit": float(w_vit)}, f, indent=2)



========== Backbone: tf_efficientnet_b3_ns (380) ==========

----- tf_efficientnet_b3_ns | Fold 1/5 -----


[tf_efficientnet_b3_ns][Fold 0] Ep 1/30 | TrF1=0.2711 VaF1=0.6923


[tf_efficientnet_b3_ns][Fold 0] Ep 2/30 | TrF1=0.5909 VaF1=0.8606


[tf_efficientnet_b3_ns][Fold 0] Ep 3/30 | TrF1=0.7223 VaF1=0.8448


[tf_efficientnet_b3_ns][Fold 0] Ep 4/30 | TrF1=0.7536 VaF1=0.8795


...

[vit_base_patch16_384][Fold 4] Ep 26/30 | TrF1=0.9952 VaF1=0.8638


[vit_base_patch16_384][Fold 4] Ep 27/30 | TrF1=0.9891 VaF1=0.8783


[vit_base_patch16_384][Fold 4] Ep 28/30 | TrF1=0.9932 VaF1=0.8656


[vit_base_patch16_384][Fold 4] Ep 29/30 | TrF1=0.9978 VaF1=0.8681


[vit_base_patch16_384][Fold 4] Ep 30/30 | TrF1=0.9970 VaF1=0.8648
>>> vit_base_patch16_384 fold F1: [0.9348120729447841, 0.9129307882550961, 0.9125193651327325, 0.8919273383952837, 0.8941923845561617] | mean=0.9093

[Auto-Weight OOF] best macro F1=0.9362 | w_eff=1.00, w_vit=0.00


In [3]:
# ============================================================
# Continue: Test 8-way TTA inference + Final Ensemble (No OCR)
# ============================================================

sub = pd.read_csv(SUB_CSV)
sub_ids = sub["ID"].astype(str).apply(lambda x: x if x.endswith(".jpg") else f"{x}.jpg")
test_files = []
for name in sub_ids:
    cands = glob.glob(os.path.join(TEST_IMG_DIR, "**", name), recursive=True)
    test_files.append(cands[0] if cands else os.path.join(TEST_IMG_DIR, name))

angles = [0, 90, 180, 270]
flips  = [False, True]

def rotate_np(img, angle):
    if angle == 0:   return img
    if angle == 90:  return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    if angle == 180: return cv2.rotate(img, cv2.ROTATE_180)
    if angle == 270: return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

backbone_test_probs = {}

for bb in BACKBONES:
    model_name = bb["name"]; img_size = bb["img_size"]
    _, tst_tf = build_transforms(img_size)
    model = create_model(model_name, bb["drop_rate"], bb["drop_path"], bb["grad_ckpt"])

    # Fold 평균 가중치 로드
    state_paths = [f"./models/{model_name}_fold{f}.pt" for f in range(NFOLDS)]
    states = [torch.load(p, map_location="cpu") for p in state_paths]
    ref_keys = list(states[0].keys())
    avg_state = {k: sum(s[k] for s in states) / len(states) for k in ref_keys}
    model.load_state_dict(avg_state, strict=True)
    model.eval()

    test_probs_accum = None
    for ang in angles:
        for fl in flips:
            imgs = []
            for p in test_files:
                img0 = np.array(Image.open(p).convert("RGB"))
                base = rotate_np(img0, ang)
                img  = cv2.flip(base, 1) if fl else base
                imgs.append(img)

            class ArrayDataset(Dataset):
                def __init__(self, arr, transform=None):
                    self.arr = arr; self.tf = transform
                def __len__(self): return len(self.arr)
                def __getitem__(self, i): return self.tf(image=self.arr[i])["image"]

            ds = ArrayDataset(imgs, tst_tf)
            loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                                num_workers=NUM_WORKERS, pin_memory=True,
                                drop_last=False, persistent_workers=False)

            probs_run = []
            for xb in tqdm(loader, desc=f"TTA {model_name} ang{ang} fl{fl}"):
                xb = xb.to(device, non_blocking=True)
                with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
                    logits = model(xb)
                    probs = torch.softmax(logits, dim=1).to(torch.float32).cpu().numpy()
                probs_run.append(probs)

            probs_run = row_normalize(np.vstack(probs_run))
            test_probs_accum = probs_run if test_probs_accum is None else (test_probs_accum + probs_run)
            torch.cuda.empty_cache()

    test_prob = row_normalize((test_probs_accum / 8.0).astype(np.float32))
    backbone_test_probs[model_name] = test_prob
    save_npz_float32(f"./test_probs/{model_name}_test_probs.npz", test_prob)
    del model; torch.cuda.empty_cache()


TTA vit_base_patch16_384 ang270 flTrue: 100%|████████████████████████████████████████████████████████████████████████████████| 197/197 [00:09<00:00, 20.31it/s]


In [4]:

# ============================================================
# Final ensemble + Expected F1 + Diagnostics
# ============================================================

# 수동 보정 (예: 강제 0.7:0.3 blend)
if w_vit < 0.1:
    print("⚠️ ViT weight too low → forcing soft blend (0.7:0.3)")
    w_eff, w_vit = 0.7, 0.3

eff_probs = np.load("./test_probs/tf_efficientnet_b3_ns_test_probs.npz")["arr_0"]
vit_probs = np.load("./test_probs/vit_base_patch16_384_test_probs.npz")["arr_0"]

ws = w_eff + w_vit
w_eff, w_vit = w_eff/ws, w_vit/ws



⚠️ ViT weight too low → forcing soft blend (0.7:0.3)


In [5]:
# Expected F1 (OOF 기반)
y_pred, y_true = [], []
for eff, vit, tgt in zip(eff_oof, vit_oof, tgt_oof):
    mix = w_eff*row_normalize(eff) + w_vit*row_normalize(vit)
    y_pred.append(mix.argmax(1))
    y_true.append(tgt)
y_pred = np.concatenate(y_pred); y_true = np.concatenate(y_true)
exp_f1 = f1_score(y_true, y_pred, average="macro")

# Test 앙상블
final_probs = row_normalize(w_eff*eff_probs + w_vit*vit_probs)
final_preds = np.argmax(final_probs, axis=1)

# 진단 리포트
with open("./debug/ensemble_diagnostics.txt", "w") as f:
    f.write(check_probs("Eff_test", eff_probs) + "\n")
    f.write(check_probs("ViT_test", vit_probs) + "\n")
    f.write(f"Pairwise corr (test probs): Eff-ViT={corrcoef(eff_probs, vit_probs):.4f}\n")
    f.write(f"Weights (normalized): eff={w_eff:.3f}, vit={w_vit:.3f}\n")
    f.write(f"Expected OOF F1: {exp_f1:.4f}\n")


In [6]:

# 제출 파일
sub["target"] = final_preds
out_name = f"sub_v9_noocr_w{w_eff:.2f}-{w_vit:.2f}_expF1{exp_f1:.4f}.csv"
sub.to_csv(out_name, index=False)

print(f"\n✅ Saved submission {out_name} (Expected F1={exp_f1:.4f})")
print("🧪 Diagnostics → ./debug/ensemble_diagnostics.txt")



✅ Saved submission sub_noocr_w0.70-0.30_expF10.9341.csv (Expected F1=0.9341)
🧪 Diagnostics → ./debug/ensemble_diagnostics.txt
